# Explanatory Data Analysis（EDA）

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/repos/signate-student-cup-2023/

/content/drive/MyDrive/repos/signate-student-cup-2023


## 1. Importing the required libraries for EDA

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Loading the data into the data frame

In [ ]:
path = Path.cwd()
train_df = pd.read_csv(path.joinpath("input", "train.csv"))
test_df = pd.read_csv(path.joinpath("input", "test.csv"))
submit_df = pd.read_csv(path.joinpath("input", "submit_sample.csv"), names=["id", "price"])

In [ ]:
train_df.head()

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158


## 3. Checking the types of data

In [ ]:
train_df.dtypes

id               int64
region          object
year             int64
manufacturer    object
condition       object
cylinders       object
fuel            object
odometer         int64
title_status    object
transmission    object
drive           object
size            object
type            object
paint_color     object
state           object
price            int64
dtype: object